## Izhikevich model of spiking neuron

$\dot{v} = 0.04 v^{2} + 5v + 140 - u + I$  
$\dot{u} = a\left( bv - u\right)$

with auxilliary after-spike reset

$v \ge 30 mV \Rightarrow \begin{cases}
    v \leftarrow c \\
    u \leftarrow u + d
\end{cases}$

where $v$ represents a membrane potential and $u$ membrane recovery variable.

**Reference**:
Izhikevich, E. M. (2003). Simple model of spiking neurons. IEEE Transactions on neural networks, 14(6), 1569-1572.

<img src="izhi_illustration.png" alt="Izhikevich model explanation" style="width: 447px; height: 303px"/>

In [1]:
# imports and other prepare stuff
from functools import partial
from IPython.display import display
import ipywidgets as widgets

from models import IzhikevichModel, IZHIKEVICH_MODEL_SLIDERS
from plotting import setup_sliders_layout, integrate_and_plot

In [2]:
grid, sliders = setup_sliders_layout(IZHIKEVICH_MODEL_SLIDERS)
# run interactive plot
ui = widgets.interactive_output(partial(integrate_and_plot, model_cls=IzhikevichModel), sliders)

display(grid, ui)

GridspecLayout(children=(Button(description='Model parameters', layout=Layout(grid_area='widget001', height='a…

Output()